# NES Graphics Extraction, Dataset Building, and VAE Generation
This notebook provides a complete workflow for extracting NES graphics tiles from ROMs, building datasets, parsing ROM headers, training and sampling with a VAE, and visualizing results. All code is ready to run and explained in detail.

## 1. Setup & Imports
Install required packages and import all necessary modules.

In [ ]:
# If needed, install missing packages (uncomment below)
# !pip install numpy pandas pillow torch scikit-learn matplotlib tqdm
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from scripts.graphics_extractor import extract_chr_data, chr_to_tiles, tiles_to_images
from scripts.DataSetBuilder import save_dataset
from scripts.RomParse import parse_nes_header
from scripts.vae_nes_tiles import build_palette, colorize_tile_from_palette, upscale, VAE

## 2. Extract Tiles from NES ROMs
Extract graphics tiles from all `.nes` files in the `roms/` folder and save them as PNG images in the `tiles/` folder.

In [ ]:
roms_dir = 'roms'
out_dir = 'tiles'
os.makedirs(out_dir, exist_ok=True)

for fname in os.listdir(roms_dir):
    if fname.lower().endswith('.nes'):
        rom_path = os.path.join(roms_dir, fname)
        try:
            chr_data = extract_chr_data(rom_path)
            tiles = chr_to_tiles(chr_data)
            save_dataset(fname.replace('.nes',''), tiles, out_dir=out_dir)
            print(f'Saved tiles for {fname}')
        except Exception as e:
            print(f'Error processing {fname}: {e}')

## 3. Visualize Extracted Tiles
Display a grid of extracted tiles from a selected ROM.

In [ ]:
rom_name = 'MicroMages'  # Change to any ROM name you have extracted
tiles_csv = f'tiles/{rom_name}_dataset.csv'
df = pd.read_csv(tiles_csv)
tile_imgs = []
for i, row in df.iterrows():
    img = Image.open(row['image_path'])
    tile_imgs.append(img)
    if i >= 63: break  # Show first 64 tiles

fig, axs = plt.subplots(8, 8, figsize=(8,8))
for idx, ax in enumerate(axs.flatten()):
    ax.imshow(tile_imgs[idx])
    ax.axis('off')
plt.suptitle(f'First 64 Tiles from {rom_name}')
plt.show()

## 4. Parse NES ROM Header
Inspect the header of a NES ROM to get PRG/CHR sizes and mapper info.

In [ ]:
rom_path = 'roms/MicroMages.nes'
header = parse_nes_header(rom_path)
print('NES ROM Header:', header)

## 5. Generate VAE Samples
Use a trained VAE model to generate new NES tiles for each cluster/class. Set the checkpoint path, number of samples, and palette options.

In [ ]:
ckpt = 'vae_checkpoint.pt'  # Path to your trained VAE .pt file
out_dir = 'vae_out'
num_samples = 16
latent = 16
num_classes = 40
palette = 'RP2C04-0004'
palette_map = [0,1,2,3]
upscale_factor = 4
cluster_id = 0

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = VAE(latent_dim=latent, conditional=True, num_classes=num_classes, emb_dim=4).to(device)
model.load_state_dict(torch.load(ckpt, map_location=device))
model.eval()

palette64 = build_palette(palette)
cluster_dir = os.path.join(out_dir, f'cluster{cluster_id:02d}')
os.makedirs(cluster_dir, exist_ok=True)

z = torch.randn(num_samples, latent).to(device)
y_sample = torch.full((num_samples,), cluster_id, dtype=torch.long).to(device)
samples = model.decode(z, y_sample)
samples = torch.clamp(samples, 0, 1)
samples = torch.round(samples * 3).long()

vae_imgs = []
for i in range(samples.size(0)):
    tile = samples[i,0].cpu().numpy()
    img = Image.fromarray(tile.astype(np.uint8), mode='L')
    img = colorize_tile_from_palette(img, palette64, palette_map)
    img = upscale(img, upscale_factor)
    tile_path = os.path.join(cluster_dir, f'tile_{i:03d}.png')
    img.save(tile_path)
    vae_imgs.append(img)
print(f'Saved {num_samples} tiles to {cluster_dir}')

## 6. Visualize VAE Generated Tiles
Display a grid of generated tiles from the VAE model.

In [ ]:
fig, axs = plt.subplots(4, 4, figsize=(8,8))
for idx, ax in enumerate(axs.flatten()):
    ax.imshow(vae_imgs[idx])
    ax.axis('off')
plt.suptitle('VAE Generated Tiles (Cluster 0)')
plt.show()

---
### End of Workflow
You now have a complete pipeline for NES graphics extraction, dataset building, VAE sampling, and visualization. You can expand this notebook with more advanced clustering, palette selection, or model training as needed.